# NASA demo template

In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML('<style>.custom-slider .bk-input-group {height: 400px;}</style>'))
display(HTML('<style>.small-custom-slider .bk-input-group {height: 200px;}</style>'))

#slider = pnw.FloatSlider(start=-0.5, end=0.5, value=0,  orientation='vertical', css_classes=["custom-slider"] )

In [2]:
import numpy as np

from bokeh.layouts import row, column, gridplot
from bokeh.models import ColumnDataSource
from bokeh.models.widgets import Slider, TextInput, Button
from bokeh.plotting import figure, output_notebook, show

output_notebook()

Loading BokehJS ...

In [3]:
from OpenVisus import *

Starting OpenVisus C:\Users\Vlaerio's PC\AppData\Local\Programs\Python\Python39\lib\site-packages\OpenVisus\__init__.py 3.9.13 (tags/v3.9.13:6de2ca5, May 17 2022, 16:36:42) [MSC v.1929 64 bit (AMD64)] sys.version_info(major=3, minor=9, micro=13, releaselevel='final', serial=0) ...


In [4]:
colormaps = ['viridis', 'plasma', 'inferno', 'magma', 'cividis','ocean', 'gist_earth', 'terrain', 'gist_stern',
             'gnuplot', 'gnuplot2', 'CMRmap', 'cubehelix', 'brg',
             'gist_rainbow', 'rainbow', 'jet', 'turbo', 'nipy_spectral',
             'gist_ncar']

In [5]:
def Assert(cond):
    if not cond:
        raise Exception("Assert failed")

class CachedDataset(PyDataset):
    
    # constructor
    def __init__(self, args):
        self.local_filename=os.path.abspath(args["local"]).replace("\\","/")
        self.remote_url=args["url"]
        self.remote_access_type = args["access"]
        self.description=args["description"]
        
        super().__init__(LoadDatasetCpp(self.remote_url))
        
        self.num_blocks = len(self.getFields()) * self.getTotalNumberOfBlocks() * len(self.getTimesteps().asVector())
        self.num_blocks_cached = 0

        self.stop_thread=False
        self.thread=None
        
        self.progress=None
        self.progress_display=None
        
    def __del__(self):
        self.stopCaching()   
        
    # createAccess
    def createAccess(self, ):
        
        access_config="""
            <access type='multiplex'>
                    <access type='disk' chmod='rw' url='file://{}' />
                    <access type='{}' url='{}' chmod="r" /> 
            </access>  
        """.format(
            self.local_filename.replace("&","&amp;"),
            self.remote_access_type,
            self.remote_url.replace("&","&amp;")) 
        
        # print("Creating access",access_config)

        access= self.createAccessForBlockQuery(StringTree.fromString(access_config))

        # at this point the cache is enabled with the new local idx file
        Assert(os.path.isfile(self.local_filename))

        return access   

    # startCaching
    def startCaching(self, background=True):
        
        if background:
            self.thread = threading.Thread(target=self.startCaching, args=(False,))
            self.stop_thread=False
            self.thread.start()        
            return 

        #print("start caching","...")
        
        access=self.createAccess()

        access.beginRead()
        
        for field in self.getFields():
            for blockid in range(self.getTotalNumberOfBlocks()): 
                for time in self.getTimesteps().asVector():
                    # print("Copying block","time",time,"field",field,"blockid",blockid,"...")
                    buffer =  self.readBlock(blockid, field=field, time=time, access=access)
                    
                     # to debug missing blocks
                    if  False and buffer is None :
                        read_block = db.createBlockQuery(blockid, ord('r'))
                        msg="# {} {} \n".format(blockid,read_block.getLogicBox().toString())
                        os.write(1, bytes(msg,'utf-8'))                   
                    
                    self.num_blocks_cached += 1
                    self.updateProgress()
                    if self.stop_thread:
                        # print("thread stopped")
                        access.endRead()
                        return
                        
        access.endRead()
        self.thread=None
        #print("caching finished done")
        
    # stopCaching
    def stopCaching(self):
        #print("stopping caching...")
        self.stop_thread=True
        if self.thread:
            self.thread.join()
            self.thread=None
    # getWidth
    def getWidth(self):
        p2=self.getLogicBox()[1]
        return p2[0]    
        
    # getHeight
    def getHeight(self):
        p2=self.getLogicBox()[1]
        return p2[1]   
        
    # getDepth
    def getDepth(self):
        p2=self.getLogicBox()[1]
        return p2[2]  
        
    # readSlice
    def readSlice(self,dir=0, slice=0,quality=-3, time=1, access=None):
        
        W,H,D=self.getWidth(), self.getHeight(), self.getDepth()
        x=[0,W] if dir!=0 else [slice,slice+1]
        y=[0,H] if dir!=1 else [slice,slice+1]
        z=[0,D] if dir!=2 else [slice,slice+1] 
        ret=self.read(x=x, y=y,z=z, quality=quality,time=time,access=access)
        
        width,height=[value for value in ret.shape if value>1]
        return ret.reshape([width,height])
        
    # readColumn
    def readXYColumn(self,Height, Depth,quality=-3, time=1, access=None):
        W,H,D=self.getWidth(), self.getHeight(), self.getDepth()
        x=[0,W]
        y=[Height,Height+1]
        z=[Depth ,Depth +1] 
        ret=self.read(x=x, y=y,z=z, quality=quality,time=time,access=access)
        #print(">",ret.shape)
        width=[value for value in ret.shape if value>1]
        return ret
        
    # setProgress
    def setProgress(self,progress, progress_display):
        self.progress=progress
        self.progress_display=progress_display   
        self.progress.min=0
        self.progress.max =self.num_blocks       

    # updateProgress
    def updateProgress(self):
                    
        if self.progress:
            self.progress.value = self.num_blocks_cached

        if self.progress_display:
            self.progress_display.value = (
                "Caching progress %.2f%% (%d/%d)" % (
                    100 * self.num_blocks_cached/self.num_blocks, 
                    self.num_blocks_cached,
                    self.num_blocks))                    

print("Utilities defined")

Utilities defined


In [6]:
NasaAtmosphericZone = []

for i in range(6):
    datasetName = "nasa-DYAMOND-atmospheric-face_"+str(i)+"_depth_52_time_1024"
    NasaAtmosphericlocal_cache="./visus-cache/"+datasetName+"/visus.idx"
    NasaAtmospheric =    {
            "url":"http://atlantis.sci.utah.edu/mod_visus?dataset="+datasetName+"&cached=1",
            "access":"network",
            "local": NasaAtmosphericlocal_cache,
            "description":'University of Utah Campus Server'
        }



    NasaAtmosphericdb=CachedDataset(NasaAtmospheric )
    NasaAtmosphericaccess=NasaAtmosphericdb.createAccess()
    W,H,D=NasaAtmosphericdb.getWidth(), NasaAtmosphericdb.getHeight(), NasaAtmosphericdb.getDepth()
    print(W,H,D)
    first_query=NasaAtmosphericdb.readSlice(dir=2, slice=(20//2)*2,access=NasaAtmosphericaccess,time=1, quality=-2)
    first_query=NasaAtmosphericdb.readSlice(dir=2, slice=48,access=NasaAtmosphericaccess,time=1, quality=-3)
    print(first_query.shape)
    first_query=NasaAtmosphericdb.readSlice(dir=1, slice=500,access=NasaAtmosphericaccess,time=1, quality=-3)
    print(first_query.shape)
    NasaAtmosphericZone.append([NasaAtmosphericdb, NasaAtmosphericaccess])



1440 1440 52
(360, 720)
(52, 720)
1440 1440 52
(360, 720)
(52, 720)
1440 1440 52
(360, 720)
(52, 720)
1440 1440 52
(360, 720)
(52, 720)
1440 1440 52
(360, 720)
(52, 720)
1440 1440 52
(360, 720)
(52, 720)


In [7]:
import colorsys
import matplotlib
from bokeh.models import LinearColorMapper, BasicTicker, ColorBar
from bokeh.palettes import *


slice = NasaAtmosphericdb.getXYSlice(resolution = 0,resample_output=False)

myPalette = linear_palette(Reds256, 128) + linear_palette(Blues256 [::-1], 128)
my_cmap1 = LinearColorMapper(palette=myPalette, low=np.min(slice), high=np.max(slice))
# my_cmap1 = LinearColorMapper(palette="Turbo256", low=np.min(slice), high=np.max(slice))
# mpl.colormaps[name]
#my_cmap1 = LinearColorMapper(palette="seismic", low=np.min(slice), high=np.max(slice))


In [8]:
import numpy as np
from bokeh.plotting import figure, show


NasaAtmosphericCurrentZone = 0
#NasaAtmosphericZone

N = 500
x = np.linspace(0, 5, N)
y = np.linspace(0, 10, N)
xx, yy = np.meshgrid(x, y)
d = np.sin(xx)*np.cos(yy)
#myImage.data_source.data  = {"image" :[d]}
counter = 0

# Here should be the code that returns an image2
def getImage(xDim,yDim):
    N = 500
    x = np.linspace(0, xDim, N)
    y = np.linspace(0, yDim, N)
    xx, yy = np.meshgrid(x, y)
    d = np.sin(xx)*np.cos(yy)
    return d
    
def getHorizontalImage(depth,time,db=NasaAtmosphericdb, access=NasaAtmosphericaccess): 
    return db.readSlice(dir=2, slice=(depth//2)*2,access=access,time=time, quality=-5)
    
def getLongImage(depth,time,db=NasaAtmosphericdb, access=NasaAtmosphericaccess): 
    #return db.readSlice(dir=0, slice=(depth//2)*2,access=access,time=time, quality=0)
    return db.readSlice(dir=0, slice=(depth//2)*2,access=access,time=time, quality=-2)
        
def getLatImage(depth,time,db=NasaAtmosphericdb, access=NasaAtmosphericaccess): 
    #return db.readSlice(dir=1, slice=(depth//2)*2,access=access,time=time, quality=0)
    return db.readSlice(dir=1, slice=(depth//2)*2,access=access,time=time, quality=-2)
    
def getHorizontalImage(depth,time,db=NasaAtmosphericZone[NasaAtmosphericCurrentZone][0], access=NasaAtmosphericZone[NasaAtmosphericCurrentZone][1]): 
    global NasaAtmosphericZone, NasaAtmosphericCurrentZone
    db=NasaAtmosphericZone[NasaAtmosphericCurrentZone][0] 
    access=NasaAtmosphericZone[NasaAtmosphericCurrentZone][1]
    return db.readSlice(dir=2, slice=(depth//2)*2,access=access,time=time, quality=-5)
    
def getLongImage(depth,time,db=NasaAtmosphericZone[NasaAtmosphericCurrentZone][0], access=NasaAtmosphericZone[NasaAtmosphericCurrentZone][1]): 
    #return db.readSlice(dir=0, slice=(depth//2)*2,access=access,time=time, quality=0)
    return db.readSlice(dir=0, slice=(depth//2)*2,access=access,time=time, quality=-2)
        
def getLatImage(depth,time,db=NasaAtmosphericZone[NasaAtmosphericCurrentZone][0], access=NasaAtmosphericZone[NasaAtmosphericCurrentZone][1]): 
    #return db.readSlice(dir=1, slice=(depth//2)*2,access=access,time=time, quality=0)
    return db.readSlice(dir=1, slice=(depth//2)*2,access=access,time=time, quality=-2)
    
dbWidth  = NasaAtmosphericdb.getWidth()
dbHeight = NasaAtmosphericdb.getHeight()
dbDepth  = NasaAtmosphericdb.getDepth()
needsRedraw = True
needToUpdatePanel = True
timeLonDepth = 1
timeLonUpdate = False
timeLatDepth = 1
timeLatUpdate = False
timeDelta = 10
timeDelta = 1

def modify_doc(doc):
    global needToUpdatePanel, timeLonDepth, timeLatDepth, timeDelta, NasaAtmosphericCurrentZone


    # Set up data
    wDim = 360
    hDim = 180
    xMin = 0
    xMax = 1440 
    yMin = 0
    yMax = 1440 
    yDim = 20
    scale = 0.5
    wDim = int((xMax-xMin)*scale)
    hDim = int((yMax-yMin)*scale)
    needToUpdatePanel = 450
    N = 200
    x = np.linspace(0, 4*np.pi, N)
    y = np.sin(x)
    source = ColumnDataSource(data=dict(x=x, y=y))

    # Set up plot
    plot = figure(plot_height=wDim, plot_width=wDim, 
                  title="Select longitude and latiutude",
                  x_axis_label='Longitude',
                  y_axis_label='Latitude',
                  x_range=[0, 1440 ], y_range=[0, 1440 ], toolbar_location=None)
    color_bar = ColorBar(color_mapper=my_cmap1, ticker= BasicTicker(),
                         location=(0,0))    
    plot.add_layout(color_bar, 'right')

    plot.xgrid[0].grid_line_color=None
    plot.ygrid[0].grid_line_color=None

# color_bar = ColorBar(color_mapper=color_mapper, ticker= BasicTicker(),
#                      location=(0,0))
    
    
    #theImage = plot.image(image=[d], x=xMin, y=yMin, dw=xMax-xMin, dh=yMax-yMin, palette="Turbo256", level="image")
    d = getHorizontalImage(20,8,db=NasaAtmosphericZone[NasaAtmosphericCurrentZone][0], access=NasaAtmosphericZone[NasaAtmosphericCurrentZone][1])
    theImage = plot.image(image=[d], x=0, y=0, dw=1440 , dh=1440 , color_mapper =my_cmap1, level="image")

    # Set up plot
    plotLong = figure(plot_height=wDim//2, plot_width=wDim, 
                      x_axis_label='Latitude',
                      y_axis_label='Depth',
                      x_range=[0, 1440], y_range=[0, 52], toolbar_location=None)
    plotLong.xgrid[0].grid_line_color=None
    plotLong.ygrid[0].grid_line_color=None

    #d = getHorizontalImage(20,8)
    e = getLongImage(500,1,db=NasaAtmosphericZone[NasaAtmosphericCurrentZone][0], access=NasaAtmosphericZone[NasaAtmosphericCurrentZone][1])
    theImageLong = plotLong.image(image=[e], 
                                  x=0, y=0, dw=1440 , dh= 52 , color_mapper = my_cmap1, level="image")

    # Set up plot
    plotLat  = figure(plot_height=wDim//2, plot_width=wDim, 
                      x_axis_label='Longitude',
                      y_axis_label='Depth',
                      x_range=[0, 1440], y_range=[0, 52], toolbar_location=None)
    plotLat.xgrid[0].grid_line_color=None
    plotLat.ygrid[0].grid_line_color=None

    #d = getHorizontalImage(20,8)
    e = getLatImage(500,1,db=NasaAtmosphericZone[NasaAtmosphericCurrentZone][0], access=NasaAtmosphericZone[NasaAtmosphericCurrentZone][1])
    theImageLat = plotLat.image(image=[e], 
                                x=0, y=0, dw=1440 , dh= 52 , color_mapper = my_cmap1, level="image")

    line_color1="#f46d43"
    line_color="#ffffff"
    line_width=3
    line_width2=1
    xSource = ColumnDataSource()
    ySource = ColumnDataSource()
    plot.line('x', 'y', source=xSource, line_width=line_width,line_color=line_color)
    plot.line('x', 'y', source=ySource, line_width=line_width,line_color=line_color)
    plot.line('x', 'y', source=xSource, line_width=line_width2,line_color=line_color1)
    plot.line('x', 'y', source=ySource, line_width=line_width2,line_color=line_color1)

    latxSource = ColumnDataSource()
    latySource = ColumnDataSource()
    plotLat.line('x', 'y', source=latxSource, line_width=line_width,line_color=line_color)
    plotLat.line('x', 'y', source=latySource, line_width=line_width,line_color=line_color)
    plotLat.line('x', 'y', source=latxSource, line_width=line_width2,line_color=line_color1)
    plotLat.line('x', 'y', source=latySource, line_width=line_width2,line_color=line_color1)

    longxSource = ColumnDataSource()
    longySource = ColumnDataSource()
    plotLong.line('x', 'y', source=longxSource, line_width=line_width,line_color=line_color)
    plotLong.line('x', 'y', source=longySource, line_width=line_width,line_color=line_color)
    plotLong.line('x', 'y', source=longxSource, line_width=line_width2,line_color=line_color1)
    plotLong.line('x', 'y', source=longySource, line_width=line_width2,line_color=line_color1)

    orientation="vertical"
    width  = 50
    height = wDim
    #display(HTML(" <style>.custom-slider .bk-input-group {height: " +str(int(wDim*.9)) +"px;}</style>" ))

    panelSize = Slider(title="size of visualization panel below", value=wDim, start=0, end=4*wDim, step=1)

    xPosition = Slider(title="Long", value=500, start=0, end=1440-1, step=1,width=width,
                       css_classes=["custom-slider"],orientation=orientation)
    yPosition = Slider(title="Lat", value=500, start=0, end=1440-1, step=1, width=width,
                       css_classes=["custom-slider"],orientation=orientation)

    depth = Slider(title="Depth", value=dbDepth-1, start=0, end=dbDepth-1, step=1,width=width,
                       css_classes=["custom-slider"],orientation=orientation)
    timeStep = Slider(title="Time", value=1, start=1, end=1024, step=1, width=width,
                       css_classes=["custom-slider"],orientation=orientation)
    
    zoneNumber = Slider(title="Zone", value=0, start=0, end=5, step=1, width=width,
                       css_classes=["custom-slider"],orientation=orientation)
    
    def setValues():
        xSource.data = dict(x=[xPosition.value,xPosition.value], y=[yMin, yMax])
        ySource.data = dict(x=[xMin, xMax], y=[yPosition.value,yPosition.value])

        latxSource.data = dict(x=[yPosition.value, yPosition.value], y=[0,2000])
        latySource.data = dict(x=[0,2000], y=[depth.value, depth.value])

        longxSource.data = dict(x=[xPosition.value, xPosition.value], y=[0,2000])
        longySource.data = dict(x=[0,2000], y=[depth.value, depth.value])
        
        
    setValues()

    
    playSlider = Slider(start=1, end=1024, value=1, step=1, title="Timestep")
    # slider.on_change('value', slider_update)

    playButton   = Button(label='► Play', width=60,button_type='success')
    playButtonS  = Button(label='► Play slow', width=60,button_type='success')
    
    playButton2   = Button(label='► Play', width=60,button_type='success')
    playButton2S  = Button(label='► Play slow', width=60,button_type='success')
    
    def animate_update(playButton=playButton):
        global timeLonDepth, timeLonUpdate, timeDelta

        #playButton.label = '❚❚ Pause (time '+str(timeLonDepth)+')'
        #theImageLong.data_source.data = {"image" :[getLongImage(xPosition.value,timeLonDepth)]}
        timeLonDepth += timeDelta
        if timeLonDepth > 1023:
            timeLonDepth = 1
        #slider.value = year
    
    def animate():
        global timeLonDepth, timeLonUpdate,timeDelta
        if playButtonS.label == '❚❚ Pause' :
            return

        if timeLonUpdate == False :
            timeLonUpdate = True 
            playButton.label = '❚❚ Pause'
            timeLonDepth = 1
            timeDelta = 10
            #doc.add_periodic_callback(animate_update, 200)
        else:
            timeLonUpdate = False 
            playButton.label = '► Play'
            #doc.remove_periodic_callback(animate_update)    
    
    playButton.on_click(animate)


    def animateS():
        global timeLonDepth, timeLonUpdate, timeDelta
        if playButton.label == '❚❚ Pause' :
            return

        if timeLonUpdate == False :
            timeLonUpdate = True 
            playButtonS.label = '❚❚ Pause'
            timeLonDepth = 1
            timeDelta = 1
            #doc.add_periodic_callback(animate_update, 200)
        else:
            timeLonUpdate = False 
            playButtonS.label = '► Play slow'
            #doc.remove_periodic_callback(animate_update)    
    
    playButtonS.on_click(animateS)
    

   
    def animate_update2(playButton2=playButton2):
        global timeLatDepth, timeLatUpdate, timeDelta

        #playButton.label = '❚❚ Pause (time '+str(timeLonDepth)+')'
        #theImageLong.data_source.data = {"image" :[getLongImage(xPosition.value,timeLonDepth)]}
        timeLatDepth += timeDelta
        if timeLatDepth > 1023:
            timeLatDepth = 1
        #slider.value = year
    
    def animate2():
        global timeLatDepth, timeLatUpdate, timeDelta
        if playButton2S.label == '❚❚ Pause' :
            return

        if timeLatUpdate == False :
            timeLatUpdate = True 
            playButton2.label = '❚❚ Pause'
            timeLatDepth = 1
            timeDelta = 10
            #doc.add_periodic_callback(animate_update, 200)
        else:
            timeLatUpdate = False 
            playButton2.label = '► Play'
            #doc.remove_periodic_callback(animate_update)    
    
    playButton2.on_click(animate2)


    def animate2S():
        global timeLatDepth, timeLatUpdate, timeDelta
        if playButton2.label == '❚❚ Pause' :
            return

        if timeLatUpdate == False :
            timeLatUpdate = True 
            playButton2S.label = '❚❚ Pause'
            timeLatDepth = 1
            timeDelta = 1
            #doc.add_periodic_callback(animate_update, 200)
        else:
            timeLatUpdate = False 
            playButton2S.label = '► Play slow'
            #doc.remove_periodic_callback(animate_update)    
    
    playButton2S.on_click(animate2S)

    # Set up callbacks
    #needToUpdatePanel = True
    def update_panel(forceUpdate = False):
        global needToUpdatePanel
        if needToUpdatePanel or forceUpdate:
            plot.plot_width      = needToUpdatePanel
            plot.plot_height     = needToUpdatePanel
            plotLong.plot_width  = needToUpdatePanel
            plotLong.plot_height = needToUpdatePanel//2
            plotLat.plot_width   = needToUpdatePanel
            plotLat.plot_height  = needToUpdatePanel//2
            #xPosition.height     = needToUpdatePanel
            display(HTML(" <style>.custom-slider .bk-input-group {height: " +str(int(needToUpdatePanel*.9)) +"px;}</style>" ))
            needToUpdatePanel    = False

    def request_update_panel(attrname, old, new):
        global needToUpdatePanel
        needToUpdatePanel = new        
        #update_panel()

    panelSize.on_change('value', request_update_panel)

    #Not using events. Just update asyncronosly
    def update_data(attrname, old, new):
        pass
        #setValues()


    #Not using events. Just update asyncronosly
#     for w in [xPosition,yPosition,offset, amplitude, phase, freq]:
#         w.on_change('value', update_data)


    #myWidgets = column(xPosition, yPosition, depth,timeStep) 
    title = Button(label="OpenViSUS streaming analysis of remote data from Utah server: 'DYAMOND c1440 llc2160, atmospheric data: U (eastward wind velocity), zone 0", button_type='primary', height = 30)
    myWidgets = row(xPosition, yPosition, depth,timeStep,zoneNumber) 
    myGrid = column(title,panelSize,
                    row(myWidgets,plot,column(row(plotLong, column(playButton,playButtonS)),
                                              playSlider
                                              , row(plotLat, column(playButton2,playButton2S))))                   )
    playSlider.disabled = True
    doc.add_root(myGrid)
    
    doc.title = "NASA demo"

    def asyncUpdate():
        global counter
        global needsRedraw, NasaAtmosphericCurrentZone 
        NasaAtmosphericCurrentZone= zoneNumber.value

        if timeLonUpdate :
            #playButton.label = '❚❚ Pause (time '+str(timeLonDepth)+')'
            playSlider.value = timeLonDepth
            theImageLong.data_source.data = {"image" :[getLongImage(xPosition.value,timeLonDepth)]}
            animate_update()

        if timeLatUpdate :
            #playButton.label = '❚❚ Pause (time '+str(timeLonDepth)+')'
            playSlider.value = timeLatDepth
            theImageLat.data_source.data = {"image" :[getLatImage(yPosition.value,timeLatDepth)]}
            animate_update2()

        # update panel size if needed
        update_panel()

        if not hasattr(asyncUpdate, "timeStepOld"):
            asyncUpdate.timeStepOld  = timeStep.value    
            asyncUpdate.xPositionOld = xPosition.value
            asyncUpdate.yPositionOld = yPosition.value
            asyncUpdate.depthOld     = depth.value
            asyncUpdate.ZoneOld      = zoneNumber.value

        #flush all events until steady
        if (asyncUpdate.timeStepOld  != timeStep.value   or    
            asyncUpdate.xPositionOld != xPosition.value  or
            asyncUpdate.yPositionOld != yPosition.value  or
            asyncUpdate.depthOld     != depth.value      or
            asyncUpdate.ZoneOld      != zoneNumber.value):
            
            asyncUpdate.timeStepOld  = timeStep.value    
            asyncUpdate.xPositionOld = xPosition.value
            asyncUpdate.yPositionOld = yPosition.value
            asyncUpdate.depthOld     = depth.value
            asyncUpdate.ZoneOld      = zoneNumber.value
            needsRedraw = True
            return
            
        if needsRedraw:
            setValues()
            theImage.data_source.data     = {"image" :[getHorizontalImage(depth.value    ,timeStep.value)]}
            theImageLong.data_source.data = {"image" :[getLongImage(xPosition.value,timeStep.value)]}
            theImageLat.data_source.data  = {"image" :[getLatImage (yPosition.value,timeStep.value)]}
            playSlider.value = timeStep.value
            NasaAtmosphericCurrentZone= zoneNumber.value
            
            
            needsRedraw = False

        counter +=1
#         print(counter, xPosition.value,yPosition.value,depth.value,timeStep.value,
#               end = '                                    \r')        
    
    doc.add_periodic_callback(asyncUpdate, 50)


In [9]:
show(modify_doc)

In [10]:
#UNCOMMENT IN CASE OF PROBLEM WITH THE PORT AND REPLACE THE "8888" BELOW WITH THE PORT NUMBER INDICATE IN THE ERROR MESSAGE
# import os 
# os.environ["BOKEH_ALLOW_WS_ORIGIN"] = "localhost:8888"
# show(modify_doc)